In [ ]:
# ⬜ Cell 0 - Disk Clearing (run if we get warning: "Disk is almost full")
!rm -rf ~/.cache/huggingface/hub
!pip cache purge -q
!find . -name "tmp_scene_*" -delete
!find . -name "scene_variants" -type d -exec rm -rf {} +
!df -h /

In [ ]:
# ⬜ Cell 1 - install base libraries
!pip -q install --upgrade transformers accelerate bitsandbytes diffusers

In [ ]:
from huggingface_hub import login

# 🔑 put in Hugging Face token
login("put in Hugging Face token")

In [ ]:
# ⬜ Cell 2 - loading Mistral-7B-Instruct-v0.2 ב-4bit


from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",          # Accelerate - deals with the mapping
    torch_dtype="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    low_cpu_mem_usage=True,
)


text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("✅ pipeline ready. device map:", getattr(model, "hf_device_map", "n/a"))

In [ ]:
# ===== Minimal helpers for the "last Cell 3" =====
# Uses globals from Cell 2: text_gen OR (model, tokenizer)

import json, textwrap, time, re
from typing import List

def llm(user_prompt: str, max_new_tokens=150, temperature=1.2, top_p=0.95, rep=1.12) -> str:
    messages = [
        {"role": "system", "content": "You write gentle, visual picture-book text for children. Avoid violence or scary content."},
        {"role": "user",   "content": user_prompt}
    ]
    chat_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    out = text_gen(
        chat_text,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=rep,
        return_full_text=False,
    )[0]["generated_text"]
    return out.strip()

def first_nonempty_line(s: str) -> str:
    for ln in s.splitlines():
        ln = ln.strip().lstrip("-*•>#").strip()
        low = ln.lower()
        if ln and not low.startswith(("assistant:", "user:", "system:", "scene ", "scene:", "1.", "2.", "3.", "4.", "5.", "###")):
            return ln
    return s.strip()

def _call_llm(prompt: str,
              max_new_tokens: int = 180,
              temperature: float = 0.9,
              top_p: float = 0.95,
              rep: float = 1.05) -> str:
    """
    Calls the LLM using objects created in תא 2.
    Prefers `text_gen` pipeline; falls back to (model, tokenizer).
    """
    if "text_gen" in globals() and text_gen is not None:
        out = text_gen(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
        return out[0]["generated_text"]

    if "model" in globals() and "tokenizer" in globals() and model is not None and tokenizer is not None:
        from transformers import TextStreamer
        import torch
        inputs = tokenizer(prompt, return_tensors="pt")
        if hasattr(model, "device"):
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        gen = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            streamer=streamer
        )
        return tokenizer.decode(gen[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    raise RuntimeError("❌ Missing backend. Run תא 2 so text_gen/model/tokenizer exist.")

def unglue_common_articles(s: str) -> str:
    """Fixes glued words like 'meadowthe' -> 'meadow the' (keeps exceptions)."""
    exceptions = {"lathe","scythe","breathe","writhe","loathe","sheathe","wreathe","clothe"}
    def repl(m):
        left, art = m.group(1), m.group(2)
        token = (left + art).lower()
        return left + art if token in exceptions else left + " " + art
    return re.sub(r"([A-Za-z]{3,})(?:(the|and|in|on|at)\b)", repl, s)

# Gentle cleanups the last Cell 3 uses
REPLACEMENTS = [
    (r"\bstriped bananas?\b", "ripe bananas"),
    (r"\bzesty lemon\b", "bright yellow"),
    (r"\brainbow fairies\b", "butterflies"),
    (r"\bglowing fairy dust\b", "soft sparkling dust"),
    (r"\bmeadowthe\b", "meadow the"),
    (r"\bpurrs quietly quietly\b", "purrs quietly"),
]

def apply_replacements(s: str) -> str:
    for pat, rep in REPLACEMENTS:
        s = re.sub(pat, rep, s, flags=re.IGNORECASE)
    return s

def fantasy_ok(title: str) -> bool:
    """Used by Cell 3 to allow a bit of 'magical' flavor in titles."""
    return re.search(r"\b(magic|magical|enchanted|spell|wizard|dragon|fairy|fairies)\b", title.lower()) is not None


In [ ]:
# ====================== Cell 3 - input, improvement, and general conistancy/solo ======================
import re, json, textwrap
from typing import List


SUBJECT_LOCK       = True    # keep the same character between scenes
SOLO_SUBJECT       = True    # excludes additional characters
START_WITH_SUBJECT = False   # if true, the sentence will start with the character

_HEAD_ALIASES = {
    "dog":   ["puppy", "pup"],  "puppy": ["dog", "pup"],
    "cat":   ["kitten", "kitty"], "kitten": ["cat", "kitty"],
    "horse": ["foal", "colt", "filly", "pony"], "foal": ["horse", "colt", "filly"],
    "bird":  ["chick"],  "child": ["boy", "girl", "kid"], "robot": []
}

def _pluralize(w: str) -> str:
    w = w.lower()
    if w.endswith("y") and len(w) > 1 and w[-2] not in "aeiou":
        return w[:-1] + "ies"
    if w.endswith(("s","x","z","ch","sh")):
        return w + "es"
    return w + "s"

def build_aliases(head: str):
    #builds a set of basic aliases from the head_noun
    h = (head or "character").lower().strip()
    alts = {h, _pluralize(h)}
    for base, vs in _HEAD_ALIASES.items():
        fam = {base, _pluralize(base), *vs, *(_pluralize(v) for v in vs)}
        if h in fam:
            alts |= fam
            break
    return alts

def normalize_subject_mentions_generic(text: str, head: str) -> str:
    """
    כל NP כמו 'a/an/the ... <alias>' (לא 'the same') → 'the same <head>'.
    גם 'another/a different <alias>' → 'the same <head>'.
    """
    aliases = "|".join(sorted(map(re.escape, build_aliases(head))))
    out = re.sub(rf"\b(another|a\s*different)\s+({aliases})\b",
                 f"the same {head}", text, flags=re.I)
    out = re.sub(rf"\b(a|an|the)\s+(?:(?!same)\w+\s+){{0,5}}({aliases})\b",
                 f"the same {head}", out, flags=re.I)
    out = re.sub(r"\s{2,}", " ", out)
    out = re.sub(r"\s+([,.!?;:])", r"\1", out)
    return out.strip()

def violates_subject_lock_generic(text: str, head: str) -> bool:
    """דוחה אם עדיין נראה שמוצגות דמויות/מופעים שונים של אותו סוג."""
    aliases = "|".join(sorted(map(re.escape, build_aliases(head))))
    if re.search(rf"\b(another|a\s*different)\s+({aliases})\b", text, flags=re.I):
        return True
    # כמה מופעי NP אחרי הנירמול? מתירים עד שניים: פתיחה + 'the same ...'
    np_hits = re.findall(rf"\b(a|an|the)\s+(?:same\s+)?({aliases})\b", text, flags=re.I)
    return len(np_hits) > 2

# ---------- Solo: animative word bank + check ----------
ANIMATE_WORDS = {
    # humans
    "man","woman","boy","girl","child","kid","person","people","crowd","vendor","seller","keeper",
    "ranger","stranger","friend","friends","family","mother","father","grandma","grandpa","teacher",
    # common animals
    "dog","dogs","puppy","puppies","cat","cats","kitten","kittens","bird","birds","horse","horses",
    "foal","foals","cow","cows","sheep","goat","goats","rabbit","rabbits","bunny","bunnies","mouse","mice",
    "bear","bears","fox","foxes","deer","duck","ducks","swan","swans","fish","fishes","frog","frogs",
    "butterfly","butterflies","bee","bees","squirrel","squirrels","dragon","dragons","wizard","wizards"
}
_DOG_BREEDS = r"(golden\s+retriever|labrador(?:\s+retriever)?|german\s+shepherd|pomeranian|poodle|beagle|bulldog|husky|malamute|akita|boxer|dachshund|doberman|greyhound|whippet|great\s+dane|chihuahua|pug|corgi|shiba|samoyed|pointer|spaniel|terrier|rottweiler|mastiff|bern(?:e|ese)\s+mountain|australian\s+shepherd|border\s+collie|bichon|shih\s*tzu|maltese|newfoundland|saint\s+bernard|weimaraner|vizsla|sheltie)"
BREED_RE = re.compile(rf"\b{_DOG_BREEDS}\b", re.I)

def violates_solo_subject(text: str, head: str) -> bool:
    """
    true if there is variation for the head_noun in the text.
    """
    low = text.lower()

    if head.lower() in {"dog","puppy","pup"} and BREED_RE.search(low):
        return True

    aliases = {a.lower() for a in build_aliases(head)}
    tokens = re.findall(r"[a-z][-\w']*", low)
    for t in tokens:
        if t in ANIMATE_WORDS and t not in aliases and t != "it":
            return True

    anim_union = "|".join(sorted(ANIMATE_WORDS))
    if re.search(rf"\b(with|and|alongside|near|beside)\s+(a|an|the)\s+(?:\w+\s+){{0,3}}({anim_union})\b", low):
        return True

    return False

# ---------- General text helpers ----------
COLOR_WORDS = {
    "red","crimson","scarlet","pink","rose","orange","amber","gold","golden","yellow","saffron",
    "green","emerald","jade","blue","azure","cerulean","navy","indigo","purple","violet","lilac",
    "white","black","brown","silver","grey","gray","rainbow"
}

def squeeze_enums(text: str, max_items=2) -> str:
    def _shrink(chunk: str) -> str:
        items = [w.strip() for w in chunk.split(",")]
        items = [i for i in items if i][:max_items]
        return ", ".join(items)
    return re.sub(r"(?:\s*,\s*[^,]{2,}){3,}", lambda m: _shrink(m.group(0)), text)

def limit_colors(text: str, max_colors=2) -> str:
    tokens, seen, out = text.split(), 0, []
    for t in tokens:
        key = t.lower().strip(",.;:!?")
        if key in COLOR_WORDS:
            if seen >= max_colors:
                continue
            seen += 1
        out.append(t)
    return " ".join(out)

def first_nonempty_line(s: str) -> str:
    for line in s.splitlines():
        line = line.strip()
        if line:
            return line
    return s.strip()

def unglue_common_articles(s: str) -> str:
    exceptions = {"lathe","scythe","breathe","writhe","loathe","sheathe","wreathe","clothe"}
    def repl(m):
        left, art = m.group(1), m.group(2)
        return left + art if (left+art).lower() in exceptions else left + " " + art
    return re.sub(r"([A-Za-z]{3,})(?:(the|and|in|on|at)\b)", repl, s)

def compact_scene(line: str) -> str:
    # take the first sentence only
    first = re.split(r'(?<=[.!?])\s+', line.strip())[0].strip()
    line = re.sub(r"\s+", " ", first)
    line = squeeze_enums(line, max_items=2)
    line = limit_colors(line, max_colors=2)
    line = re.sub(r"\s*,\s*(?=[.!?]|$)", "", line)
    words = line.split()
    if len(words) > 28:
        line = " ".join(words[:28]).rstrip(",") + "."
    if not re.search(r'[.!?]$', line):
        line += "."
    return line

# Generalized wording fix
REPLACEMENTS = [
    (r"\bstriped bananas?\b", "ripe bananas"),
    (r"\bzesty lemon\b", "bright yellow"),
    (r"\bglowing fairy dust\b", "soft sparkling dust"),
    (r"\bmeadowthe\b", "meadow the"),
    (r"\bpurrs quietly quietly\b", "purrs quietly"),
]
def deweird(s: str) -> str:
    out = s
    for pat, rep in REPLACEMENTS:
        out = re.sub(pat, rep, out, flags=re.IGNORECASE)
    out = re.sub(r"\bpaw(ing)? at (a )?(crimson|scarlet|red|orange|yellow|green|blue|purple)\b",
                 r"paw\1 at a \3 leaf", out, flags=re.IGNORECASE)
    return out

EAT_FIXES = [
    (r"\b(nibbl\w*|munch\w*|snack\w*) on (water lily pads|lily pads|petals|flowers|blossoms|leaves)\b",
     r"sniffs the \2"),
    (r"\b(nibbl\w*|munch\w*|snack\w*) on (clover blossoms|raspberries?)\b",
     r"sniffs the \2"),
]
def deweird_eating(s: str) -> str:
    out = s
    for pat, rep in EAT_FIXES:
        out = re.sub(pat, rep, out, flags=re.IGNORECASE)
    return out

def is_fragment_end(s: str) -> bool:
    bad_ends = {"the","a","an","and","or","of","to","in","on","at","with","from","by","into","over","under","near","while"}
    last = re.sub(r"[^A-Za-z]+$", "", s.strip().lower().split()[-1])
    return last in bad_ends

def fix_trailing_fragment(s: str) -> str:
    s = re.sub(r"\.{3,}$", ".", s)
    s = re.sub(r"\s*\.\.+\s*$", ".", s)
    if is_fragment_end(s):
        cut = re.split(r'(?<=[.!?])\s+', s.strip())
        if cut and re.search(r'[.!?]$', cut[0]): s = cut[0]
        else:
            words = s.strip().split()
            while words and is_fragment_end(" ".join(words)):
                words = words[:-1]
            s = " ".join(words)
    if not re.search(r'[.!?]$', s.strip()):
        s = s.rstrip(",;:") + "."
    return s

def fantasy_ok(title: str) -> bool:
    return re.search(r"\b(magic|magical|enchanted|spell|wizard|dragon|fairy|fairies)\b", title.lower()) is not None

# ---------- Reading the language model from the last cell ----------
def _call_llm(prompt: str, max_new_tokens=160, temperature=0.9, top_p=0.9, rep=1.08) -> str:
    if "llm" in globals() and callable(globals()["llm"]):
        return globals()["llm"](prompt, max_new_tokens=max_new_tokens,
                                temperature=temperature, top_p=top_p, rep=rep)
    if "text_gen" in globals():
        try:
            out = text_gen(prompt, max_new_tokens=max_new_tokens,
                           do_sample=True, temperature=temperature, top_p=top_p,
                           return_full_text=False)
            return out[0]["generated_text"]
        except TypeError:
            out = text_gen(prompt, max_new_tokens=max_new_tokens,
                           do_sample=True, temperature=temperature, top_p=top_p)
            txt = out[0]["generated_text"]
            return txt[len(prompt):].lstrip() if txt.startswith(prompt) else txt
    if "model" in globals() and "tokenizer" in globals():
        inputs = tokenizer.apply_chat_template(
            [{"role":"user","content":prompt}],
            add_generation_prompt=True, return_tensors="pt"
        ).to(model.device)
        gen = model.generate(
            inputs, max_new_tokens=max_new_tokens,
            do_sample=True, temperature=temperature, top_p=top_p,
            repetition_penalty=rep, eos_token_id=tokenizer.eos_token_id
        )
        return tokenizer.decode(gen[0, inputs.shape[1]:], skip_special_tokens=True)
    raise RuntimeError("❌ No language model found. Run Cell 2 before running Cell 3")

# ---------- שיפור כותרת/דמות ----------
_JSON_OBJ = re.compile(r"\{.*?\}", re.S)

def improve_seed(topic: str, char_desc: str, tries=3):
    for _ in range(tries):
        prompt = textwrap.dedent(f"""
            Improve the following children's story seed.
            Return ONLY compact JSON: {{"title": "...", "character": "..."}}
            - Keep it child-friendly.
            - Title: 3–7 words, no quotes.
            - Character: one concise line (≤ 16 words), present tense.

            Topic: "{topic}"
            Character: "{char_desc}"
        """).strip()
        raw = _call_llm(prompt, max_new_tokens=120)
        m = _JSON_OBJ.search(raw)
        if m:
            try:
                j = json.loads(m.group(0))
                title = (j.get("title") or topic).strip().strip('"')
                char  = (j.get("character") or char_desc).strip().strip('"')
                if title and char:
                    return title, char
            except Exception:
                pass
    return topic.strip(), char_desc.strip()

# ---------- חילוץ ביטוי קנוני + שם עצם ----------
def canonicalize_identity(topic: str, char_desc: str):
    prompt = textwrap.dedent(f"""
        From the following children's-story seed, extract a short canonical identity phrase for the MAIN CHARACTER.
        Return ONLY JSON:
        {{"canon":"<2–6 words noun phrase to refer to the character, may include a name>", "noun":"<head noun>"}}

        Topic: "{topic}"
        Character: "{char_desc}"
    """).strip()
    raw = _call_llm(prompt, max_new_tokens=80, temperature=0.2, top_p=0.9)
    canon, noun = None, None
    m = _JSON_OBJ.search(raw)
    if m:
        try:
            j = json.loads(m.group(0))
            canon = j.get("canon")
            noun  = j.get("noun")
        except Exception:
            pass
    if not canon:
        words = re.findall(r"[A-Za-z][-\w']*", char_desc)
        canon = " ".join(words[:6]) if words else "main character"
    if not noun:
        m = re.search(r"\b(boy|girl|child|kid|kitten|cat|puppy|dog|foal|horse|robot|fox|bear|rabbit|mouse|dragon|wizard|bird)\b",
                      (char_desc + " " + canon).lower())
        noun = m.group(1) if m else "character"
    return canon.strip().strip('"').replace("  ", " "), noun.strip()

def ensure_canon_once(s: str, canon_phrase: str, *, at_begin: bool = False) -> str:
    """
    makes sure the canonical phrase only appears once
    """
    pat = re.compile(re.escape(canon_phrase), re.I)
    if pat.search(s):
        s = pat.sub("__SUBJ__", s, count=1)
        s = pat.sub("", s)
        return s.replace("__SUBJ__", canon_phrase)

    if at_begin:
        return f"{canon_phrase} — {s.lstrip()}"

    if re.search(r'[.!?]\s*$', s):
        return re.sub(r'\s*[.!?]\s*$', f" — {canon_phrase}.", s)
    else:
        return f"{s} — {canon_phrase}"

# ---------- יצירת סצנה אחת ----------
def ask_scene(title: str, char_desc: str, canon_phrase: str, head_noun: str,
              used: List[str], attempt=0):
    min_words = 10 if attempt == 0 else 8
    forbid = " | ".join(used) if used else "none"
    realism = "" if fantasy_ok(title) else "• Keep it realistic; no fantasy or mythical creatures.\n"

    inclusion = (f'• Include this subject phrase ONCE at the beginning: "{canon_phrase}" (do NOT change species/name/number).'
                 if START_WITH_SUBJECT else
                 f'• Include this subject phrase EXACTLY ONCE (anywhere): "{canon_phrase}" (do NOT change species/name/number).')

    prompt = f"""
Write EXACTLY ONE sentence for a child-friendly picture-book scene for “{title}”.
• 18–28 words, present tense, concrete visuals (who/where/action).
• Use at most TWO colour words; avoid long comma lists.
{realism}{inclusion}
• Single subject only (one {head_noun}); no duplicates, no crowds.
• No other characters at all — no people and no other animals or {head_noun}s; only scenery and simple objects (ball, leaf, pond, tree, bench, path).
• Do NOT start with “Scene”, numbers, or bullets.
• Avoid repeating phrases from: {forbid}
• End cleanly with a period. Then write <END> on its own line.
• Keep the SAME subject across scenes: if you mention it again, write “the same {head_noun}” or use a pronoun — never “another” or “a different” {head_noun}.

Main character: {char_desc}
Scene:
""".strip()

    raw = _call_llm(prompt, max_new_tokens=120, temperature=0.95, top_p=0.92, rep=1.12)
    text = raw.split("<END>")[0].strip()
    line = first_nonempty_line(text)

    # cleaning and polishing
    line = compact_scene(line)
    line = unglue_common_articles(line)
    line = deweird(line)
    line = deweird_eating(line)
    line = ensure_canon_once(line, canon_phrase, at_begin=START_WITH_SUBJECT)
    line = fix_trailing_fragment(line)

    # keeping the same subject through scenes
    if SUBJECT_LOCK:
        line = normalize_subject_mentions_generic(line, head=head_noun)
        if violates_subject_lock_generic(line, head=head_noun):
            return None

    # exclude extra characters
    if SOLO_SUBJECT and violates_solo_subject(line, head_noun):
        return None

    if len(line.split()) >= min_words and all(line.lower() != s.lower() for s in used):
        return line
    return None

def generate_scenes(title: str, char_desc: str, canon_phrase: str, head_noun: str, tries=8):
    used, scenes = [], []
    for _ in range(5):
        line = None
        for attempt in range(tries):
            line = ask_scene(title, char_desc, canon_phrase, head_noun, used, attempt=attempt)
            if line:
                used.append(line); scenes.append(line); break
        if not line:
            raise ValueError("The model failed to make 5 proper scenes after a number of tries")
    return scenes

# ---------- user input, improvement, canonization and locking ----------
topic_input = input("📘 Topic (few words): ").strip()
char_input  = input("🎭 Main character (describe): ").strip()

story_topic = topic_input or "A Colourful Tale"
character_description = char_input or "a playful kitten"

title, character_description = improve_seed(story_topic, character_description)
canon_phrase, head_noun     = canonicalize_identity(title, character_description)

# locking the head_noun
canon_phrase = f"the {head_noun}".strip()

print(f"🔹 Title: {title}")
print(f"🔹 Character: {character_description}")
print(f"🔹 Canon phrase: {canon_phrase}  (noun: {head_noun})")

scenes = generate_scenes(title, character_description, canon_phrase, head_noun)

print("\n✅ 5 Scenes:")
for i, s in enumerate(scenes, 1):
    print(f"{i}. {s}")


In [ ]:
# ============== Cell 4: reference and character identification (OWL-ViT Large→Base→CLIP→mid) ==============
import os, re, torch, numpy as np
from PIL import Image, ImageDraw
from diffusers import StableDiffusionPipeline
from IPython.display import display

assert all(k in globals() for k in ["character_description","canon_phrase","head_noun"]), \
    "❌ Run Cell 3 first - missing character_description/canon_phrase/head_noun"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- עזרי שמירה/קרופ ----------
def square_from_box(img, box, pad=1.15, out=256):
    W,H = img.size
    x1,y1,x2,y2 = map(float, box)
    cx,cy = (x1+x2)/2,(y1+y2)/2
    side = max(x2-x1, y2-y1) * pad
    nx1,ny1 = max(0,int(cx-side/2)), max(0,int(cy-side/2))
    nx2,ny2 = min(W,int(cx+side/2)), min(H,int(cy+side/2))
    crop = img.crop((nx1,ny1,nx2,ny2))
    return crop.resize((out,out), Image.LANCZOS), (nx1,ny1,nx2,ny2)

# make reference image

ref_image = None
"""
if use_reference:
    if files:
        print("📤 העלה תמונה (jpg/png):")
        up = files.upload()
        if up: ref_image = Image.open(list(up.keys())[0]).convert("RGB")
    else:
        p = input("🔎 נתיב לקובץ מקומי: ").strip()
        if p and os.path.exists(p): ref_image = Image.open(p).convert("RGB")
"""
if ref_image is None:
    print("🖌️ creating reference image")

    if device == "cuda":
        torch.cuda.empty_cache()


    sd_dtype = torch.float16 if device == "cuda" else torch.float32
    ref_pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=sd_dtype,
        safety_checker=None
    ).to(device)
    auto_prompt = (
        f"{canon_phrase}, close-up portrait, head and shoulders, looking at camera, "
        f"centered, storybook illustration, soft natural lighting, high quality, clean background"
    )
    neg = "cropped, cut-off, out of frame, profile, side view, multiple subjects, watermark, text, lowres, blurry"
    gen = torch.Generator(device=device).manual_seed(12345)
    ref_image = ref_pipe(prompt=auto_prompt, negative_prompt=neg,
                         num_inference_steps=40, guidance_scale=8.0,
                         generator=gen).images[0]
    # ⚠️ erase and clean
    del ref_pipe
    if device == "cuda": torch.cuda.empty_cache()

# ---------- 1) OWL-ViT: Free text character recognition ----------
def owl_detect(img, labels, prefer_large=True):
    from transformers import pipeline
    model_id = "google/owlvit-large-patch14" if prefer_large else "google/owlvit-base-patch32"
    det = pipeline("zero-shot-object-detection", model=model_id,
                   device=0 if device=="cuda" else -1)
    # improve accuracy by resize for the long side to 896
    W,H = img.size
    scale = 896 / max(W,H)
    base = img.resize((int(W*scale), int(H*scale)), Image.LANCZOS) if scale<1.0 else img
    out = det(base, candidate_labels=labels)
    if not out: return None
    # give the one with the best score, resize back to original size
    best = max(out, key=lambda d:d["score"])
    if best["score"] < 0.15: return None # סף זיהוי
    b = best["box"]; s = best["score"]
    if scale<1.0:
        inv = 1/scale
        box = (b["xmin"]*inv, b["ymin"]*inv, b["xmax"]*inv, b["ymax"]*inv)
    else:
        box = (b["xmin"], b["ymin"], b["xmax"], b["ymax"])
    return (box, s, model_id)

noun = (head_noun or "character").strip()
canon_clean = re.sub(r'^(a|an|the)\s+','', canon_phrase, flags=re.IGNORECASE)
labels = []
for t in [canon_clean, noun, f"{noun} face", f"{noun} head", f"{noun} portrait",
          f"{noun} full body", "main character", "single character"]:
    t = t.strip()
    if t and t.lower() not in {x.lower() for x in labels}: labels.append(t)

full_512 = ref_image.resize((512,512), Image.LANCZOS)

box, score, mid = None, None, None
try:
    r = owl_detect(full_512, labels, prefer_large=True)
    if r and r[1] >= 0.20: box, score, mid = r
    else:
        r = owl_detect(full_512, labels, prefer_large=False)
        if r and r[1] >= 0.18: box, score, mid = r
except Exception as e:
    print("⚠️ details: CLIP-crop unavailable, try OWL-ViT", e)
    method = None

# ---------- 2) fail: multi-scale CLIP-crop ----------
if box is None:
    try:
        import open_clip
        model, _, preprocess = open_clip.create_model_and_transforms("ViT-L-14", pretrained="openai")
        model = model.to(device).eval()
        tok = open_clip.get_tokenizer("ViT-L-14")
        def enc_t(t):
            t = re.sub(r"\s+"," ", t).strip()[:140]
            with torch.no_grad():
                toks = tok([t]).to(device)
                v = model.encode_text(toks); return v / v.norm(dim=-1, keepdim=True)
        def enc_i(pil):
            with torch.no_grad():
                ii = preprocess(pil).unsqueeze(0).to(device)
                v = model.encode_image(ii); return v / v.norm(dim=-1, keepdim=True)
        tfeat = enc_t(f"close-up {noun} face" if noun else "close-up face")
        W,H = full_512.size; best = (-1,(0,0,224,224),224)
        for cs in (176,208,240,272,304):
            base = full_512
            if W < cs or H < cs:
                scale = max(cs/W, cs/H)
                base = full_512.resize((int(W*scale)+2, int(H*scale)+2), Image.LANCZOS)
            BW, BH = base.size
            for y in range(0, BH - cs + 1, 16):
                for x in range(0, BW - cs + 1, 16):
                    patch = base.crop((x,y,x+cs,y+cs))
                    sc = torch.cosine_similarity(enc_i(patch), tfeat).item()
                    if sc > best[0]: best = (sc,(x,y,x+cs,y+cs),cs)
        box, score, mid = best[1], best[0], "open_clip"
    except Exception as e:
        print("⚠️ CLIP-crop unavailable", e)

# ---------- 3) last fail ----------
method = None
if box is None:
    W,H = full_512.size
    side = int(min(W,H)*0.36)
    cx,cy = W//2, int(H*0.42)
    box = (cx-side, cy-side, cx+side, cy+side)
    method = "center"
else:
    method = f"{mid} ({score:.3f})"

# ---------- 4) save ----------
ref_focus, used_box = square_from_box(full_512, box, pad=1.15, out=256)
full_512.save("reference_full.png")
ref_focus.save("reference_focus.png")

viz = full_512.copy()
draw = ImageDraw.Draw(viz)
draw.rectangle(tuple(map(int, used_box)), outline=(255,0,0), width=4)

print(f"✅ נשמרו קבצי רפרנס — שיטת קרופ: {method}")
print("   • reference_full.png (512×512)")
print("   • reference_focus.png (256×256)")
display(viz)
display(ref_focus)

In [ ]:
!pip -q install git+https://github.com/openai/CLIP.git

In [ ]:
# ====================== Cell 5 — SD + IP-Adapter + Text-first Ranking (Ref-boosted grid) ======================
import os, re, json, random, shutil
from pathlib import Path
from PIL import Image
import numpy as np
import torch

# ---------- Setup ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

assert "scenes" in globals() and isinstance(scenes, list) and len(scenes) == 5, "❌ missing scenes from Cell 3"
title  = globals().get("title",  "Untitled Story")
canon_phrase = globals().get("canon_phrase", "the character")
head_noun    = globals().get("head_noun", "character")

# Reference image (from Cell 4)
if "ref_image" not in globals() or ref_image is None:
    if os.path.exists("ref_image.png"):
        ref_image = Image.open("ref_image.png").convert("RGB")
    else:
        raise RuntimeError("❌ missing reference image")

# Style / negatives (fallbacks if not defined)
STYLE = globals().get("STYLE",
    "storybook illustration, soft natural light, vibrant yet gentle colors, clean background, high quality"
)
NEG_COMMON = globals().get("NEG_COMMON",
    "lowres, blurry, noisy, watermark, text, cut-off, cropped, out of frame, duplicate, multiple subjects"
)

# ---------- Pipeline with IP-Adapter ----------
from diffusers import StableDiffusionPipeline
if "pipe" not in globals() or pipe is None:
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch_dtype,
        safety_checker=None
    ).to(device)
    print("✅ SD v1.5 pipeline loaded.")

if not hasattr(pipe, "_ip_adapter_loaded"):
    try:
        pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")
        pipe._ip_adapter_loaded = True
        print("✅ IP-Adapter loaded for SD1.5.")
    except Exception as e:
        print("⚠️ Failed to load IP-Adapter:", e)

def set_ip_adapter_scale(pipe, scale: float):
    if hasattr(pipe, "set_ip_adapter_scale"):
        pipe.set_ip_adapter_scale(scale)
    elif hasattr(pipe, "ip_adapter_scale"):
        pipe.ip_adapter_scale = scale
    else:
        raise RuntimeError("IP-Adapter scale API not found on this pipeline build.")

# ---------- Embedding backends for ranking ----------
oc_model = oc_preprocess = oc_tokenizer = None
clip_model = clip_preprocess = None
try:
    import open_clip
    oc_model, _, oc_preprocess = open_clip.create_model_and_transforms(
        "ViT-H-14", pretrained="laion2b_s32b_b79k", device=device
    )
    oc_model.eval()
    oc_tokenizer = open_clip.get_tokenizer("ViT-H-14")
    print("✅ open_clip (ViT-H-14) ready.")
except Exception:
    try:
        import clip
        clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
        clip_model.eval()
        print("✅ CLIP ViT-B/32 ready.")
    except Exception as e:
        raise RuntimeError("❌ Neither open_clip nor CLIP available.") from e

@torch.inference_mode()
def encode_image_feat(img: Image.Image):
    if oc_model is not None:
        t = oc_preprocess(img).unsqueeze(0).to(device)
        feat = oc_model.encode_image(t).float()
    else:
        t = clip_preprocess(img).unsqueeze(0).to(device)
        feat = clip_model.encode_image(t).float()
    return feat / feat.norm(dim=-1, keepdim=True)

@torch.inference_mode()
def encode_text_feat(text: str):
    s = re.sub(r"\s+", " ", text.strip())
    if oc_model is not None:
        tokens = oc_tokenizer([s]).to(device)
        feat = oc_model.encode_text(tokens).float()
    else:
        import clip as _clip
        tokens = _clip.tokenize([s]).to(device)
        feat = clip_model.encode_text(tokens).float()
    return feat / feat.norm(dim=-1, keepdim=True)

ref_feat = encode_image_feat(ref_image) if ref_image is not None else None

# ---------- Ranking: text-first, ref tie-break ----------
@torch.inference_mode()
def rank_text_first(text: str, paths, ref_feat=None, k=4):
    """
    1) Rank all by text↔image; take top-k by text.
    2) Among top-k, pick highest by ref↔image (if ref exists), else best-by-text.
    Returns (best_path, all_text_scores_list).
    """
    tfeat = encode_text_feat(text)
    scored = []
    for p in paths:
        img = Image.open(p).convert("RGB")
        emb = encode_image_feat(img)
        s_txt = torch.matmul(emb, tfeat.T).item()
        scored.append((s_txt, p, emb))

    scored.sort(key=lambda x: x[0], reverse=True)
    top = scored[:min(k, len(scored))]
    if not top:
        return (max(scored)[1] if scored else None), [s for s,_,_ in scored]

    if ref_feat is None:
        return top[0][1], [s for s,_,_ in scored]

    best_p, best_sr = top[0][1], -1e9
    for s_txt, p, emb in top:
        s_ref = torch.matmul(emb, ref_feat.T).item()
        if s_ref > best_sr:
            best_sr, best_p = s_ref, p
    return best_p, [s for s,_,_ in scored]

# ---------- Scales & layout (Ref-boosted grid) ----------
def ip_scale_base(shot):
    # Slightly stronger than the softer variant to let identity read, still scene-friendly
    return 0.18 if shot == "wide" else 0.26

def variant_grid(base, shot):
    """
    Build a small grid of (ip_adapter_scale, guidance_scale) with a gentle ref boost.
    Wider shots get a lower cap; medium/portrait can push a bit higher.
    """
    cap = 0.32 if shot == "wide" else 0.42    # shot-aware cap
    hi  = min(cap, base * 1.35)               # higher identity candidate
    mid = min(cap, (base + hi) / 2)           # mid candidate

    return [
        (0.06,                    7.5),  # near text-only
        (max(0.10, base*0.7),     7.5),
        (base,                    7.8),
        (mid,                     7.8),
        (hi,                      8.0),  # gentle ref boost
    ]

def decide_plan(scene_text: str):
    st = scene_text.lower()
    shot = "wide" if any(k in st for k in ["field","park","meadow","street","shore","sky","hills","valley","landscape"]) else "medium"
    orient = "landscape" if any(k in st for k in ["wide","horizon","hills","valley","street","shore","sea","sky"]) else "portrait"
    return shot, orient

def size_for(orient: str):
    if orient == "landscape":
        return 768, 576
    elif orient == "portrait":
        return 576, 768
    else:
        return 640, 640

# ---------- Generate & select ----------
random.seed(1234); np.random.seed(1234)
os.makedirs("scene_variants", exist_ok=True)
os.makedirs("scene_best", exist_ok=True)

best_scores = {}
all_best = []

for idx, scene in enumerate(scenes, 1):
    scene_short = re.sub(r"\s+", " ", scene).strip()
    shot, orient = decide_plan(scene_short)
    width, height = size_for(orient)
    base = ip_scale_base(shot)
    grid = variant_grid(base, shot)

    # Ranking text uses ONLY the scene sentence (no style/shot tokens)
    rank_text = scene_short
    sd_prompt = f"{scene_short}, {STYLE}"
    neg = NEG_COMMON

    gen_paths = []
    for k_idx, (ip_sc, guidance) in enumerate(grid, 1):
        try:
            set_ip_adapter_scale(pipe, ip_sc)
        except Exception as e:
            raise RuntimeError(f"IP-Adapter scale set failed: {e}")

        seed = 1000*idx + k_idx
        generator = torch.Generator(device=device).manual_seed(seed)

        img = pipe(
            prompt=sd_prompt,
            negative_prompt=neg,
            ip_adapter_image=ref_image,
            num_inference_steps=28,
            guidance_scale=guidance,
            width=width, height=height,
            generator=generator
        ).images[0]

        outp = Path("scene_variants") / f"scene{idx:02d}_v{k_idx:02d}.png"
        img.save(outp)
        gen_paths.append(str(outp))

    # Text-first ranking; break ties by reference among top-k
    best_path, text_scores = rank_text_first(rank_text, gen_paths, ref_feat=ref_feat, k=4)
    best_scores[idx] = float(max(text_scores)) if text_scores else float("nan")

    dest = Path("scene_best") / f"scene_{idx}.png"
    shutil.copy2(best_path, dest)
    all_best.append(str(dest))
    print(f"✅ Scene {idx}: chose {Path(best_path).name}  | text-score≈{best_scores[idx]:.4f}")
    plt.imshow(Image.open(best_path)); plt.axis("off"); plt.title(f"Best for Scene {idx}"); plt.show()

# Save simple metrics
with open("best_scores.json", "w", encoding="utf-8") as f:
    json.dump(best_scores, f, ensure_ascii=False, indent=2)

print("\n🎉 Done. Best images saved in scene_best/.  Ranking = text-first (ref tie-break).")


In [ ]:
# ====================== Cell6 - export the story (HTML + JSON) ======================
import os, json, html, base64, re
from datetime import datetime
from pathlib import Path
from PIL import Image
import torch

# --- Preconditions ---
assert "title" in globals() and "character_description" in globals(), "❌ חסרים title/character_description."
assert "scenes" in globals() and isinstance(scenes, list) and len(scenes) == 5, "❌ missing scenes"
assert os.path.isdir("scene_best"), "❌ run Cell 5"

# --- Reference image: save / locate ---
ref_path = None
try:
    if "ref_image" in globals() and ref_image is not None:
        ref_image = ref_image.convert("RGB")
        ref_path = "ref_image.png"
        ref_image.save(ref_path)
    elif os.path.exists("ref_image.png"):
        ref_path = "ref_image.png"
except Exception as e:
    print("⚠️ Failed to save ref_image.png:", e)
    ref_path = "ref_image.png" if os.path.exists("ref_image.png") else None

# --- JSON meta export ---
meta = {
    "title": title,
    "character": character_description,
    "canon_phrase": canon_phrase,
    "head_noun": head_noun,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "rank_backend": RANK_BACKEND if "RANK_BACKEND" in globals() else "n/a",
    "best_scores": best_scores if "best_scores" in globals() else {},
    "scenes": [
        {"index": i, "text": scenes[i-1], "image": f"scene_best/scene_{i}.png"}
        for i in range(1, 6)
    ],
    "created_at": datetime.utcnow().isoformat() + "Z"
}
if ref_path:
    meta["ref_image"] = ref_path

with open("story.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

# --- HTML document ---
def esc(x): return html.escape(x, quote=True)

html_body = []
html_body.append(f"<h1>{esc(title)}</h1>")
html_body.append(f"<p><b>Main character:</b> {esc(character_description)}</p>")

# Reference image BEFORE Scene 1
if ref_path:
    html_body.append(f"""
    <figure class="reference">
      <img src="{esc(ref_path)}" alt="reference image" />
      <figcaption>Reference image</figcaption>
    </figure>
    """.strip())

# Scenes 1..5
for i in range(1, 6):
    txt = scenes[i-1]
    img = f"scene_best/scene_{i}.png"
    score = (best_scores.get(i) if 'best_scores' in globals() else None)
    html_body.append(f"""
    <div class="scene">
      <h2>Scene {i}{' — CLIP {:.3f}'.format(score) if (score is not None) else ''}</h2>
      <p>{esc(txt)}</p>
      <img src="{esc(img)}" alt="scene {i}" />
    </div>
    """.strip())

html_doc = f"""
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8" />
<title>{esc(title)}</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; margin: 24px; }}
h1 {{ margin-bottom: 0.2rem; }}
p {{ line-height: 1.5; }}
.reference {{ margin: 18px 0 28px; text-align: center; }}
.reference img {{ max-width: 100%; height: auto; border-radius: 12px; box-shadow: 0 6px 24px rgba(0,0,0,0.12); }}
.reference figcaption {{ font-size: 0.9rem; color: #666; margin-top: 6px; }}
.scene {{ margin: 28px 0; }}
.scene img {{ max-width: 100%; height: auto; border-radius: 12px; box-shadow: 0 6px 24px rgba(0,0,0,0.15); }}
footer {{ margin-top: 24px; color: #777; font-size: 0.9rem; }}
</style>
</head>
<body>
{''.join(html_body)}
<footer>
  <p>Generated on {esc(meta['created_at'])}</p>
</footer>
</body>
</html>
""".strip()

with open("storybook.html", "w", encoding="utf-8") as f:
    f.write(html_doc)

# --- Single-file HTML: embed ALL images (ref + scenes) as base64 data URIs ---
def embed_image(html_text: str, path: str) -> str:
    """Replace src='path' with a data: URI, if file exists."""
    if not (path and os.path.exists(path)):
        print(f"⚠️ Missing image for embedding: {path}")
        return html_text
    ext = Path(path).suffix.lower().lstrip(".")
    mime = "image/png" if ext in ("png", "") else ("image/jpeg" if ext in ("jpg", "jpeg") else f"image/{ext}")
    with open(path, "rb") as imgf:
        b64 = base64.b64encode(imgf.read()).decode("ascii")
    return html_text.replace(f'src="{path}"', f'src="data:{mime};base64,{b64}"')

# Read the generated HTML
with open("storybook.html", "r", encoding="utf-8") as f:
    single = f.read()

# Embed scenes
for i in range(1, 6):
    single = embed_image(single, f"scene_best/scene_{i}.png")

# Embed reference (if present)
if ref_path:
    single = embed_image(single, ref_path)

# Remove any <base> if present (not used here, but safe)
single = re.sub(r'<base[^>]*>', '', single, flags=re.I)

with open("storybook_single.html", "w", encoding="utf-8") as f:
    f.write(single)

print("✅ Saved files:")
print(" • story.json")
print(" • storybook.html  (uses relative image files)")
print(" • storybook_single.html  (all images embedded)")
if ref_path:
    print(f" • {ref_path}")
